In [3]:
from secret_key import openai_key
import os
os.environ['OPENAI_API_KEY'] = openai_key

from langchain_community.llms import OpenAI
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

llm = OpenAI(temperature = 0.6)

In [4]:
prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open an {cuisine} restaurant, suggest me a name for that. Give only 1 name"
)

name_chain = LLMChain(llm = llm, prompt = prompt_template_name, output_key = 'restaurant_name')

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = "Suggest 10 menu items for {restaurant_name}. give it as a comma seperatead list."
)

food_items_chain = LLMChain(llm = llm, prompt = prompt_template_items, output_key = 'menu_items')

In [5]:
chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name','menu_items']
)

chain({'cuisine': 'Arabic'})

{'cuisine': 'Arabic',
 'restaurant_name': '\n\n"Al-Jannah Cuisine"',
 'menu_items': '\n\n1. Shawarma platter\n2. Falafel wrap\n3. Hummus and pita appetizer\n4. Lamb kebab skewers\n5. Vegetarian moussaka\n6. Chicken shish tawook\n7. Baklava dessert\n8. Spinach and feta borek\n9. Beef kofta plate\n10. Lentil soup'}

In [6]:
from langchain.agents import AgentType, initialize_agent, load_tools

tools = load_tools(["wikipedia", "llm-math"],llm = llm)

agent = initialize_agent(
    tools,
    llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

agent.run("When was elon musk born and what is his age in 2023")

'Elon Musk was born on June 28, 1971 and will be 52 years old in 2023.'

In [7]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm = llm, prompt = prompt_template_name, memory = memory)
name = chain.run("Mexican")
print(name)



"Taco Fiesta"


In [8]:
name = chain.run("Indian")
print(name)



"Spice Junction"


In [10]:
print(chain.memory.buffer)

Human: Mexican
AI: 

"Taco Fiesta"
Human: Indian
AI: 

"Spice Junction"


In [11]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm = OpenAI(temperature = 0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [12]:
convo.run("Who won the first cricket world cup")

" The first Cricket World Cup was held in 1975 and was won by the West Indies team. The tournament was a round-robin format, with eight teams competing. The final match was between West Indies and Australia, with West Indies winning by 17 runs. The player of the tournament was West Indies' Clive Lloyd, who scored 333 runs throughout the tournament."

In [13]:
convo.run("what is 5+5")

' The answer to 5+5 is 10.'

In [14]:
convo.run("who was the captain of the winning team")

' The captain of the winning team, West Indies, was Clive Lloyd. He led the team to victory and was also named the player of the tournament.'

In [16]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup
AI:  The first Cricket World Cup was held in 1975 and was won by the West Indies team. The tournament was a round-robin format, with eight teams competing. The final match was between West Indies and Australia, with West Indies winning by 17 runs. The player of the tournament was West Indies' Clive Lloyd, who scored 333 runs throughout the tournament.
Human: what is 5+5
AI:  The answer to 5+5 is 10.
Human: who was the captain of the winning team
AI:  The captain of the winning team, West Indies, was Clive Lloyd. He led the team to victory and was also named the player of the tournament.


In [17]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm = OpenAI(temperature = 0.6),
    memory = memory
)

convo.run("Who won the first cricket world cup")

" The first cricket world cup was won by the West Indies in 1975. They defeated Australia in the final by 17 runs. The tournament was held in England and was organized by the International Cricket Council (ICC). The West Indies team was led by Clive Lloyd and had players such as Viv Richards, Gordon Greenidge, and Andy Roberts. The final was played at Lord's Cricket Ground in London."

In [18]:
convo.run("what is 5+5")

' 5+5 is equal to 10.'

In [19]:
convo.run("who was the captain of the winning team")

' I am not sure which team you are referring to, but if you are talking about the recent Super Bowl, the captain of the winning team was Tom Brady. He plays for the Tampa Bay Buccaneers and has won seven Super Bowls in his career. Is there a specific team or sport you were asking about?'